In [2]:
from wilds import get_dataset
import argparse
import os

import pandas as pd
import torch
import torchvision.transforms as transforms
from torch.optim import SGD
from wilds import get_dataset

from spuco.datasets import GroupLabeledDatasetWrapper, WILDSDatasetWrapper
from spuco.evaluate import Evaluator, GroupEvaluator
from spuco.group_inference import EIIL
from spuco.robust_train import GroupDRO
from spuco.models import model_factory
from spuco.utils import Trainer, set_seed
from spuco.utils.misc import get_model_outputs

dataset = get_dataset(dataset="waterbirds", download=True, root_dir="/data")

transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        ])


train_data = dataset.get_subset(
    "train",
    transform=transform
)

val_data = dataset.get_subset(
    "val",
    transform=transform
)

In [3]:
valset = WILDSDatasetWrapper(dataset=val_data, metadata_spurious_label="background", verbose=True)

Partitioning data indices into groups: 100%|██████████| 1199/1199 [00:00<00:00, 1963674.54it/s]


In [12]:
import random 

subset_indices = []

for key in valset.group_partition.keys():
    subset_indices.extend(random.sample(valset.group_partition[key], k=3 * int(len(valset.group_partition[key]) / 5)))

In [13]:
import pickle 

with open("wbirds_5pct_val_set.pkl", "wb") as f:
    pickle.dump(subset_indices, f)

In [14]:
small_valset =  WILDSDatasetWrapper(dataset=val_data, metadata_spurious_label="background", verbose=True, subset_indices=subset_indices)

Partitioning data indices into groups: 100%|██████████| 1199/1199 [00:00<00:00, 250996.73it/s]


In [15]:
total_len = 0

for key in small_valset.group_partition.keys():
    total_len += len(small_valset.group_partition[key])
    
print(total_len)

714
